## IBM Applied Data Science Capstone


This notebook mainly deals with the capstone project carried out as a part of IBM Applied Data Science course covering the following topics:

    -FourSquare API
    -Clustering
    -Reporting based on the above results
    

In [1]:
import pandas as pd
# from geopy.geocoders import OpenCage
from opencage.geocoder import OpenCageGeocode
import geocoder
import time
import folium

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [3]:
wiki_data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
wiki_data.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [4]:
neighborhood_data = pd.DataFrame(columns = ['Postal code','Borough', 'Neighborhood'], data = wiki_data[wiki_data.Borough!='Not assigned'].values)
neighborhood_data.Neighborhood = neighborhood_data.Neighborhood.replace({'/':','},regex=True)
# neighborhood_data[neighborhood_data.Neighborhood=='NaN'].count()
neighborhood_data.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [5]:
neighborhood_data.shape

(103, 3)

In [6]:
geolocator = OpenCageGeocode("d9151bc21e3c47daa72ac27bf5422c76")

# for i,data in neighborhood_data.iterrows():
#     lat_lng = None
#     while(lat_lng==None):
#         location = geocoder.google('{}, Toronto, Ontario'.format(data['Postal code']))
# #         latitude = location[0]['geometry']['lat']
# #         longitude = location[0]['geometry']['lng']
#     neighborhood_data['Latitude'] = location.latlng[0]
#     neighborhood_data['Longitude'] = location.latlng[1]
#     print(i, data['Postal code'])
#     time.sleep(1.0)
    
neighborhood_data.head()


,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [39]:
lat_lng = pd.read_csv('./Geospatial_Coordinates.csv')

In [43]:
lat_lng.head()
lat_lng.set_index('Postal Code', inplace=True)

Postal Code
M3A    43.753259
M4A    43.725882
M5A    43.654260
M6A    43.718518
M7A    43.662301
         ...    
M8X    43.653654
M4Y    43.665860
M7Y    43.662744
M8Y    43.636258
M8Z    43.628841
Name: Latitude, Length: 103, dtype: float64

In [46]:
neighborhood_data['Latitude'] = lat_lng.loc[neighborhood_data['Postal code'], 'Latitude'].values
neighborhood_data['Longitude'] = lat_lng.loc[neighborhood_data['Postal code'], 'Longitude'].values
neighborhood_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [47]:
location = geolocator.geocode("Toronto", country="CA")
latitude_CA = location[0]['geometry']['lat']
longitude_CA = location[0]['geometry']['lng']
print("The latitude and longitude of Toronto, CA are {}, {}".format(latitude_CA, longitude_CA))

The latitude and longitude of Toronto, CA are 43.6534817, -79.3839347


In [50]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude_CA, longitude_CA], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhood_data['Latitude'], neighborhood_data['Longitude'], neighborhood_data['Borough'], neighborhood_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto
